Данные для обучения и тестирования
=====================================
Чтобы оценить, насколько хорошо наши обучаемые модели обобщают, мы можем разделить наши данные на обучающий и тестовый наборы:

<img src="figures/train_test_split.svg" width="100%">


In [ ]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X, y = iris.data, iris.target

classifier = KNeighborsClassifier()

Если подумать о том, как обычно осуществляется машинное обучение, идея разделения обучения и тестирования имеет смысл. Системы реального мира обучаются на имеющихся у них данных, и по мере поступления других данных (от клиентов, датчиков или других источников) обученный классификатор должен прогнозировать результат на основе принципиально новых данных. Мы можем смоделировать это во время обучения, используя разделение обучение/тест: тестовые данные представляют собой симуляцию «будущих данных», которые поступят в систему во время реальной работы.

Особенность набора iris в том, что метки в iris отсортированы, а это означает, что если мы разделим данные, используя пропорциональное разделение, мы получим все определенные метки (0 и 1) и очень мало других (2). Мы хотим разделить, как показано выше, но после того, как данные будут случайным образом перетасованы.

In [ ]:
y

Чтобы получить точную симуляцию реального мира, мы перетасуем наши данные, а затем разделим их. Используем генератор случайных чисел для формирования случайной последовательности индексов для перемешивания при копировании.

In [ ]:
import numpy as np
rng = np.random.RandomState(0)

permutation = rng.permutation(len(X))
permutation

In [ ]:
permutation
X, y = X[permutation], y[permutation]
print(y)

Теперь нам нужно разделить данные на обучающие и тестирующие. К счастью, это распространенный шаблон в машинном обучении, и в scikit-learn есть встроенная функция для разделения данных по этапам обучения и тестирования. Если доля  разделения не указана, то получаем 75% данных для обучения и 25% для тестирования. 80% и 20% — еще одно распространенное разделение, для указания распределения задаётся либо доля обучающих данных `train_size=0.8`, либо тестовых `test_size=0.2`. но здесь нет жестких правил. По умолчанию порядок образцов случайным образом перемешивается (если это не нужно, используйте параметр `shuffle=False`). Самое главное — честно оценить вашу систему на данных, которых она не видела во время обучения!

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8)
print("Labels for training and testing data")
print(train_y.shape, test_y.shape)
print(train_y)
print(test_y)

Обратите внимание, что данные ещё раз перетасовались!

Оценивая производительность классификатора на данных, используемых во время обучения, можно получить ложную уверенность в правильности модели предсказания. Это может привести к запуску в эксплуатацию системы, которая не сможет предсказать новые данные! Гораздо лучше использовать разделение данных для обучения и тестирования, чтобы правильно увидеть, как обученная модель работает с новыми данными.

In [ ]:
classifier.fit(train_X, train_y)
pred_y = classifier.predict(test_X)
print("Fraction Correct")
print(np.sum(pred_y == test_y) / float(len(test_y)))

Мы также можем визуализировать правильные и неудавшиеся прогнозы.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
correct_idx = np.where(pred_y == test_y)[0]
print(correct_idx)
incorrect_idx = np.where(pred_y != test_y)[0]
print(incorrect_idx)

In [ ]:
# Plot two dimensions
colors = ["darkblue", "darkgreen", "gray"]
for n, color in enumerate(colors):
    idx = np.where(test_y == n)[0]
    plt.scatter(test_X[idx, 0], test_X[idx, 1], color=color, label="Class %s" % str(n))
plt.scatter(test_X[incorrect_idx, 0], test_X[incorrect_idx, 1], color="darkred")
# Make xlim larger to accommodate legend
plt.xlim(3, 9)
plt.legend(loc=3)
plt.title("Iris Classification results")
plt.show()

Мы видим, что ошибки возникают в области перекрытия зеленого (класс 1) и серого (класс 2). Это дает нам представление о том, какие признаки добавить — любой признак, который помогает разделить класс 1 и класс 2, должен улучшить производительность классификатора.